# Interpretation of Electroencephalography Measurements for Human Movements

w207 Summer 2015, Final Project

By Michael Marks, Nihar Patel, Carson Forter, Ji Yan, Jeff Yau

![live brain](http://s1.ibtimes.com/sites/www.ibtimes.com/files/styles/v2_article_large/public/2015/04/25/brain.jpg?itok=jIf_rHLW)

## Introduction
Electroencephalography (EEG) is a non-invasive method of measuring brain activity. An array of sensors are arranged on a person's scalp. These sensors detect the electrical activity of the neurons firing immediately under the skull. Though this technology has limited spatial resolution and cannot detect electrical activity within deep brain structures, it has the advantage of having high temporal resolution; it measures electrical activity on the time scale that biological processes in the brain actually happen. This technology has many medical uses, one of which is allowing for a brain-to-computer interface. In particular, because the human brain controls movement in part with surface-oriented neurons, there is potential for using EEG to interpret muscle movements as signals from the brain. Advances in sensor hardware will allow for greater spatial resolution in the future, but the current challenge is in interpreting the raw brain signals into the intended muscle movement.

On June 29th, 2015, Kaggle, a data science competition platform, introduced [a contest](https://www.kaggle.com/c/grasp-and-lift-eeg-detection) for EEG classification. The dataset was a series of EEG records from volunteers asked to repeatedly perform a task with their right hand. Each hand movement was recorded on video, and carefully documented into a matching dataset, in effect creating a set of raw sensor data matched with data of what arm movement corresponds to that data. Competitors were asked to use the sample dataset to produce the algorithm that most accurately predicts the correct arm movement from the raw data.

In this workbook, we describe how our group approached the problem. We outline our preliminary research, our initial tests, how we handled the large dataset, and finally our results.

## Libraries Used
We primarily use the [scikit-learn](http://scikit-learn.org/stable/index.html) python library for this project.

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import time
import random
import gc

# SK-learn libraries.

# scipy
from scipy.signal import butter, lfilter



## Dataset
The data can be downloaded from Kagle [here](https://www.kaggle.com/c/grasp-and-lift-eeg-detection/data). This workbook uses the folder structure data\test\ and data\train. Each file contains data recorded at 500 Hz, so there are 500  rows of data for every second recorded. For each participant, there are seperate files for each session containing the recordings from 32 EEG sensors. Parallel to each of these files is a file describing which of 6 different arm motions are being performed at any given frame (1/500th of a second).

In [11]:
# Location of data can be changed here.
data_path = r'data/'

### Loading data
Here we define a function for loading a subset of data into a Pandas dataframe. This is necessary because the data is too large to hold in memory on an average desktop computer, and so we will need to call this function many times on different chunks of the data.

In [12]:
def load_data(subjects_to_use=[1,2],series_for_training=[1-3]):
    # Make a list of all the filenames with training data.
    train_data_filenames = glob.glob(data_path + "/subj" + subjects_to_use +
                                     "_series"+series_for_training + "*data.csv")

    # Initialize an empty dataframe.
    train_data= pd.DataFrame()
    
    # Load the dataframe with the contents of each file.
    for file_ in train_data_filenames:
        train_data = train_data.append(pd.read_csv(file_,index_col=None, header=0))
    
    # Make a list of all the filenames with training labels.
    train_labels_filesnames = glob.glob(data_path + "/subj" + subjects_to_use +
                                     "_series"+series_for_training + "*events.csv")
    
    # Initialize an empty dataframe.
    train_labels = pd.DataFrame()
    
    #Load the dataframe with the contents of each file.
    for file_ in train_labels_filenames:
        train_labels = train_labels.append(pd.read_csv(file_,index_col=None, header=0))

    # Return the resulting two dataframes.
    return train_data, train_labels

## Pre-Processing/Feature Engineering
Our group spent the vast majority of our time researching the best way to pre-process the data. We attempted  feature engineering by transforming the raw data using both a priori knowledge and general data reduction techniques. We began by consulting with medical EEG researchers and reading literature on EEG processing.

**Removing Channels**

Our medical EEG consultant did not have experience with interpreting muscle movement data, but was able to share some insights into neuroanatomy. The following brain map shows the different functional regions of the brain:

![brain functional regions](http://lh3.ggpht.com/_RIjx_Mg4ZVM/TNbSn_XhNcI/AAAAAAAACeY/Abc73jPpSbU/image_thumb6.png)

We anticipated that the frontal lobe of the brain would contain the relevant information for our problem. As the source of muscle movements, we belived that the region called the primary motor cortex is particularly important for predicting arm movements. The next image is a representation of what the primary motor cortex controls:

![homunculus](http://brainconnection.brainhq.com/wp-content/uploads/2013/03/1b.gif)

This is a coronal section of the brain (as if you were looking face-to-face with xray vision). The surface and about an inch under it, shaded in salmon color in the picture, is called the cortex. EEG sensors pick up the electrical firings of cells on the surface as shaded in blue, but note that the surface has grooves called gyri where the EEG sensors would have an especially hard time telling the difference between adjacently active sections of the cortex. The motor cortex, responsible for innervating our muscles, is almost always represented in the same spot and in the same order for humans, and we learned that individual differences likely wouldn't be picked up with the spatial resolution available using EEG. For example, if Sensor 1 is positioned above the motor strip at the position where the fingers are represented, we would expect that sensor to be active when the participants are closing and releasing their fingers from the object.

Another important detail we learned was that the left side of the brain controls the right half of the body, so the cells we expect to be firing are on the left-side motor cortex.

We produced a function to remove channels that we anticipated would contain more noise than information.

In [13]:
# Remove the channels we don't want 
def Remove_Channels(df):
    df.drop(df.columns[[15,16,20,21,22,25,26,27,28,29,30,31,32]], axis=1, inplace=True)
    return df

**Frequency Filtering**

We identified a paper that highlighted the oscillatory nature of nerual network electrical activity. It suggested that for interpreting motor neuron activation, EEG sensor data in the 8-12 Hz and 16-24 Hz range should be used.

We used the following bandpass filter function to eliminate frequencies in the raw sensor data outside of the desired ranges.

In [ ]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

**Signal Smoothing**

The Kaggle dataset description explains that the labels can be off by +/- 75 ms. Also, we observed that the raw data has frequent dips and rises during both positive and negative conditions. Consequently, we decided that some method of compressing the signal temporarily and smoothing would be beneficial. This requires care because the competition rules state that future data cannot be used to predict earlier labels.

We produced a number of functions designed to compress and smooth the signal.

In [15]:
# Bin the time.
def Bin_Time(num_rows,num_bins):
    Bin_Size = num_rows/num_bins
    Bins = np.zeros(shape=(num_rows,num_bins))
    Bin_Min = 0
    Bin_Max = Bin_Size
    for i in range(0,num_bins):
        Bins[Bin_Min:Bin_Max,i] = 1
        Bin_Min = Bin_Min + Bin_Size
        Bin_Max = Bin_Max + Bin_Size
    return Bins

# Return a rolling metric of each column in a pandas dataframe with a given window size. Returns df of same size. 
# Metric can be mean, var,min, max, skew, or kurt.
def df_rolling_metric(df,window=2, metric='mean'):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        Roll_Array = getattr(pd, ('rolling_' + metric))(df.iloc[0:,i],window,min_periods = 0).fillna(0)
        list_.append(Roll_Array)
    new_df = pd.concat(list_,1)
    for i in range(0,Num_Cols):
        new_df=new_df.rename(columns = {i:metric + str(i)})
    return np.array(new_df.astype('float32'))

# Return rolling quantile of each column in a pandas dataframe with a given window and quantile. Returns df of same size. 
def df_rolling_quantile(df,window,quantile):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        Roll_Array = pd.rolling_quantile(df.iloc[0:,i],window,min_periods = 0).fillna(0)
        list_.append(Roll_Array)
    new_df = pd.concat(list_,1)
    for i in range(0,Num_Cols):
        new_df=new_df.rename(columns = {i:'Pct'+ quantile + "_" + str(i)})
    return np.array(new_df.astype('float32'))


# BE CAREFUL NOT TO SUPPLY TOO MANY COLUMNS TO THIS FUNCTION. Returns 2^N columns, where N = intitial columns. 
# Return rolling pairwise correlation of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_corr(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        list_.append(pd.rolling_corr(df.iloc[0:,i],window,min_periods = 0))
    return pd.concat(list_,1)



# BE CAREFUL NOT TO SUPPLY TOO MANY COLUMNS TO THIS FUNCTION. Returns 2^N columns, where N = intitial columns. 
# Return rolling pairwise correlation of each column in a pandas dataframe with a given window. Returns df of same size. 
def df_rolling_cov(df,window):
    list_ = []
    Num_Cols = len(df.columns)
    for i in range(0,Num_Cols):
        list_.append(rolling_cov(df.iloc[0:,i],window,min_periods = 0))
    return pd.concat(list_,1)

# Return the rolling root of the means of the squares of each value.
# This is used to calculate the power of a signal.
def window_rms(a, window_size):
    a2 = np.power(a,2)
    window = np.ones(window_size)/float(window_size)
    return np.sqrt(np.convolve(a2, window, 'valid'))

**Feature Reduction**

We were concerned about the size of the dataset. We could not fit all rows of data in memory, and this problem was compounded by our feature extraction methods that would produce extra columns in the data set. We decided to try some methods of feature reduction to alleviate the memory issues.

In [ ]:
#run PCA and return the number of PCs that explain the given amount of variance. 
def extractFeatures_PCA(Train_Data,Test_Data, PercentVarExplained):
    pca = PCA()
    pca.fit(Train_Data)  
    
    Explained_Variance_Ratios = pca.explained_variance_ratio_
    for i in range(1,len(Explained_Variance_Ratios)):
        if sum(Explained_Variance_Ratios[0:i]) >= PercentVarExplained:
                   NumPCs = i + 1 #add 1 since numpy array ranges are not inclusive
                   break
    return np.float32(pca.transform(Train_Data)[:,0:NumPCs]),np.float32(pca.transform(Test_Data)[:,0:NumPCs])


## Testing Features
After creating features and data reduction functions, we created a pipeline for testing these functions with a simple linear regression model. Through this process, we discovered that we would constantly observe around 98% accuracy; roughly 98% of labels were negative, and roughly only 2% of all labels in the dataset are positive. This is a reflection of quick motions that participants were asked to perform, and their relatively short duration compared to time at rest.

We decided to judge the performance of our pre-processing steps using AUC scoring. **JEFF EXPLAIN HERE**